# 사고 지점 크롤링
- selenium 세팅 필요

In [4]:
import os, glob
from pathlib import Path
import time
import datetime
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

문제점
- 중단하고 재실행할 때 tmp폴더 (./test)폴더에 옮겨지지 못한 이미지가 있으면 오작동함. - 비우도록 작동해야함
- 중단하고 재실행할 때 목표 파일이 이미 존재할 때 오류가 발생함 - 건너 뛰거나, 덮어쓰게 해야함.
- ~2017년도, 즉 2018년 1월 이전에는 광명시 모양이 살짝 달랐고, 그 때문에 2017년도부터 지도의 정렬 기준이 살짝 달라져 다시 좌표 맞춰줘야함...

In [ ]:
fig_folder = "./figs"

cwd = Path.cwd()

tmp_path = "./tmp"
os.makedirs(tmp_path, exist_ok=True)


def delay_send(self, key, delay=0.3):
    self.send_keys(key)
    time.sleep(delay)

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
prefs = {
    "profile.default_content_settings.popups": 0, # 팝업 차단 해제
    "download.prompt_for_download": False,  # 다운로드 대화상자 비활성화
    "download.default_directory": str(cwd / tmp_path),  # 다운로드 폴더 지정
    "savefile.default_directory": str(cwd / tmp_path),
    "profile.default_content_setting_values.automatic_downloads": 1,
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.

driver.get('chrome://settings/')
time.sleep(1)
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.5);')
time.sleep(1)
driver.get('https://taas.koroad.or.kr/locGovPo/lae/sta/chr/main.do')

time.sleep(5)


button = driver.find_element(By.ID, 'tabGis')
button.click()
time.sleep(2)

# 사고부분 셀렉트
accident_labels = [f"accident{i}" for i in range(16)]
folder_labels = [
    "전체",
    "차대사람",
    "승합차",
    "이륜차",
    "택시",
    "신호위반",
    "화물차",
    "렌터카",
    "승용차",
    "차량단독",
    "시내버스",
    "고령운전자",
    "자전거",
    "원동기장치자전거",
    "음주운전",
    "사업용자동차",
]

start_times = [
    datetime.date(year, month, 1) for year in range(2007, 2024) for month in range(1, 13)
]



def init_datebox():
    date_box = driver.find_element(By.ID, "startDate")

    key_list = [
        Keys.SPACE,
        Keys.TAB,
        Keys.TAB,
        Keys.ENTER,
        Keys.RIGHT,
        Keys.RIGHT,
        Keys.RIGHT,
        Keys.DOWN,
        Keys.DOWN,
        Keys.ENTER,
        Keys.ENTER,
    ]
    for k in key_list:
        delay_send(date_box, k)


#"startDate"
def move_datebox_a_month_ago(id, day_shift=0):
    day_key_list = []
    if day_shift > 0:
        day_key_list = [Keys.RIGHT for _ in range(day_shift)]
    date_box = driver.find_element(By.ID, id)
    key_list = [
        Keys.SPACE,
        Keys.TAB,
        Keys.TAB,
        Keys.ENTER,
        Keys.LEFT,
        Keys.ENTER,
        *day_key_list,
        Keys.ENTER,
    ]
    for k in key_list:
        delay_send(date_box, k)

first_image = True

current_end_day = 31
end_time = datetime.date(2023, 12, 31)
next_end_day = -1

need_init = True
try_max = 10

for start_time in start_times[::-1]:
    if need_init:
        init_datebox()
        need_init = False
    else:
        move_datebox_a_month_ago("startDate")
        day_shift = max(next_end_day - current_end_day, 0)
        current_end_day = next_end_day
        move_datebox_a_month_ago("endDate", day_shift)
    
    sido_combo = driver.find_element(By.ID, "gisSidoCombo")
    delay_send(sido_combo, "경기도")
    sigungu_combo = driver.find_element(By.ID, "gisSigunguCombo")
    delay_send(sigungu_combo, "광명시")
    search_button = driver.find_element(By.XPATH, f"//button[@onclick='lae.sta.lta.searchAccident(1);']")
    
    

    for for_label, folder_label in zip(accident_labels, folder_labels):
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.2)
        label_checkbox = driver.find_element(By.XPATH, f"//label[@for='{for_label}']")
        label_checkbox.click()
        time.sleep(0.25)
        delay_send(search_button, Keys.ENTER, delay=8)
        
        num_try = 0 
        while True:
            try:
                time.sleep(1)
                image_element = driver.find_element(By.XPATH, "//div/div/img[contains(@src, 'LAYERS=GNRL_GIS')]")
                image_src = image_element.get_attribute("src")
                if first_image:
                    image_element.screenshot(f"{folder_label}_{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.png")
                    first_image = False
            except:
                num_try += 1
                if num_try >= try_max:
                    assert False
                continue
            else:
                break
        
        
        driver.execute_script(f'window.open("{image_src}");')
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        driver.execute_script(f'''
        var a = document.createElement('a');
        a.href = "{image_src}";
        a.download = '';
        document.body.appendChild(a);
        a.click();
        ''')

        time.sleep(8)

        dst_folder = f"{fig_folder}/{folder_label}"
        num_try = 0 
        while True:
            try:
                time.sleep(10)
                img_path = glob.glob(f"{tmp_path}/*.png")[0]
                dst_name = f"{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.png"
            except:
                num_try += 1
                if num_try >= try_max:
                    assert False
                continue
            else:
                break

        os.makedirs(dst_folder, exist_ok=True)
        os.rename(img_path, f"{dst_folder}/{dst_name}")

        time.sleep(1)
        driver.close()  #링크 이동 후 탭 닫기
        time.sleep(0.25)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(random.randint(2, 10)+30*random.choice([0, 0, 0, 0, 1]))
    
    end_time = start_time - datetime.timedelta(days=1)
    next_end_day = int(end_time.strftime("%d"))


time.sleep(5)
driver.quit()

SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' on line 18 (3613517921.py, line 20)

# 기타 잡 코드

# 드라이버 켜진상태로 터졌을때 이어가기

In [11]:
start_time, for_label

(datetime.date(2021, 7, 1), 'accident13')

In [ ]:
resume_start_time = start_time
resume_label = for_label
driver.switch_to.window(driver.window_handles[0])

try_max = 10

for start_time in start_times[::-1]:
    if resume_start_time and resume_start_time != start_time:
        continue
    elif resume_start_time == start_time:
        resume_start_time = False
    else:
        resume_start_time = False
        if need_init:
            init_datebox()
            need_init = False
        else:
            move_datebox_a_month_ago("startDate")
            day_shift = max(next_end_day - current_end_day, 0)
            current_end_day = next_end_day
            move_datebox_a_month_ago("endDate", day_shift)
    
    sido_combo = driver.find_element(By.ID, "gisSidoCombo")
    delay_send(sido_combo, "경기도")
    sigungu_combo = driver.find_element(By.ID, "gisSigunguCombo")
    delay_send(sigungu_combo, "광명시")
    search_button = driver.find_element(By.XPATH, f"//button[@onclick='lae.sta.lta.searchAccident(1);']")
    
    

    for for_label, folder_label in zip(accident_labels, folder_labels):
        if resume_label and resume_label != for_label:
            continue
        else:
            resume_label = False
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.2)
        label_checkbox = driver.find_element(By.XPATH, f"//label[@for='{for_label}']")
        label_checkbox.click()
        time.sleep(0.25)
        delay_send(search_button, Keys.ENTER, delay=2)
        #delay_send(label_checkbox, Keys.ENTER)
        num_try = 0 
        while True:
            try:
                time.sleep(1)
                image_element = driver.find_element(By.XPATH, "//div/div/img[contains(@src, 'LAYERS=GNRL_GIS')]")
                image_src = image_element.get_attribute("src")
                if first_image:
                    image_element.screenshot(f"{folder_label}_{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.png")
                    first_image = False
            except:
                num_try += 1
                if num_try >= try_max:
                    assert False
                continue
            else:
                break
        #urllib.request.urlretrieve(image_src, f"text/{label}.png")
        
        driver.execute_script(f'window.open("{image_src}");')
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        driver.execute_script(f'''
        var a = document.createElement('a');
        a.href = "{image_src}";
        a.download = '';
        document.body.appendChild(a);
        a.click();
        ''')

        dst_folder = f"./figs/{folder_label}"
        num_try = 0 
        while True:
            try:
                time.sleep(10)
                img_path = glob.glob(f"{tmp_path}/*.png")[0]
                dst_name = f"{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.png"
            except:
                num_try += 1
                if num_try >= try_max:
                    assert False
                continue
            else:
                break

        os.makedirs(dst_folder, exist_ok=True)
        os.rename(img_path, f"{dst_folder}/{dst_name}")

        time.sleep(1)
        driver.close()  #링크 이동 후 탭 닫기
        time.sleep(0.25)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(random.randint(2, 8)+20*random.choice([0, 0, 0, 0, 0, 1]))
    
    end_time = start_time - datetime.timedelta(days=1)
    next_end_day = int(end_time.strftime("%d"))


time.sleep(5)
driver.quit()

In [16]:
start_time, for_label

(datetime.date(2007, 1, 1), 'accident15')

# 처음부터 다시하되 특정 지점부터 시작하기

In [ ]:
# 다시 시작할 지점 설정
resume_start_time = datetime.date(2017, 12, 1)
resume_label = "accident0"

fig_folder = "./figs_test"

cwd = Path.cwd()

tmp_path = "./tmp"
os.makedirs(tmp_path, exist_ok=True)


def delay_send(self, key, delay=0.3):
    self.send_keys(key)
    time.sleep(delay)

options = webdriver.ChromeOptions()
options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
prefs = {
    "profile.default_content_settings.popups": 0, # 팝업 차단 해제
    "download.prompt_for_download": False,  # 다운로드 대화상자 비활성화
    "download.default_directory": str(cwd / tmp_path),  # 다운로드 폴더 지정
    "savefile.default_directory": str(cwd / tmp_path),
    "profile.default_content_setting_values.automatic_downloads": 1,
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)  # Optional argument, if not specified will search path.

driver.get('chrome://settings/')
time.sleep(1)
driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.5);')
time.sleep(1)
driver.get('https://taas.koroad.or.kr/locGovPo/lae/sta/chr/main.do')

time.sleep(5)


button = driver.find_element(By.ID, 'tabGis')
button.click()
time.sleep(2)

# 사고부분 셀렉트
accident_labels = [f"accident{i}" for i in range(16)]
folder_labels = [
    "전체",
    "차대사람",
    "승합차",
    "이륜차",
    "택시",
    "신호위반",
    "화물차",
    "렌터카",
    "승용차",
    "차량단독",
    "시내버스",
    "고령운전자",
    "자전거",
    "원동기장치자전거",
    "음주운전",
    "사업용자동차",
]

start_times = [
    datetime.date(year, month, 1) for year in range(2007, 2024) for month in range(1, 13)
]



def init_datebox():
    date_box = driver.find_element(By.ID, "startDate")

    key_list = [
        Keys.SPACE,
        Keys.TAB,
        Keys.TAB,
        Keys.ENTER,
        Keys.RIGHT,
        Keys.RIGHT,
        Keys.RIGHT,
        Keys.DOWN,
        Keys.DOWN,
        Keys.ENTER,
        Keys.ENTER,
    ]
    for k in key_list:
        delay_send(date_box, k, delay=0.15)


#"startDate"
def move_datebox_a_month_ago(id, day_shift=0):
    day_key_list = []
    if day_shift > 0:
        day_key_list = [Keys.RIGHT for _ in range(day_shift)]
    date_box = driver.find_element(By.ID, id)
    key_list = [
        Keys.SPACE,
        Keys.TAB,
        Keys.TAB,
        Keys.ENTER,
        Keys.LEFT,
        Keys.ENTER,
        *day_key_list,
        Keys.ENTER,
    ]
    for k in key_list:
        delay_send(date_box, k, delay=0.15)

first_image = True

current_end_day = 31
end_time = datetime.date(2023, 12, 31)
next_end_day = -1

need_init = True
try_max = 10

for start_time in start_times[::-1]:
    if need_init:
        init_datebox()
        need_init = False
    else:
        move_datebox_a_month_ago("startDate")
        day_shift = max(next_end_day - current_end_day, 0)
        current_end_day = next_end_day
        move_datebox_a_month_ago("endDate", day_shift)

    if not isinstance(resume_start_time, type(None)) and resume_start_time != start_time:
        end_time = start_time - datetime.timedelta(days=1)
        next_end_day = int(end_time.strftime("%d"))
        continue
    else:
        resume_start_time = None
    
    sido_combo = driver.find_element(By.ID, "gisSidoCombo")
    delay_send(sido_combo, "경기도")
    sigungu_combo = driver.find_element(By.ID, "gisSigunguCombo")
    delay_send(sigungu_combo, "광명시")
    search_button = driver.find_element(By.XPATH, f"//button[@onclick='lae.sta.lta.searchAccident(1);']")
    
    first_image = True

    for for_label, folder_label in zip(accident_labels, folder_labels):
        if not isinstance(resume_label, type(None)) and resume_label != for_label:
            continue
        else:
            resume_label = None
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(0.2)
        label_checkbox = driver.find_element(By.XPATH, f"//label[@for='{for_label}']")
        label_checkbox.click()
        time.sleep(0.25)
        delay_send(search_button, Keys.ENTER, delay=8)
        
        num_try = 0 
        while True:
            try:
                time.sleep(1)
                image_element = driver.find_element(By.XPATH, "//div/div/img[contains(@src, 'LAYERS=GNRL_GIS')]")
                image_src = image_element.get_attribute("src")
                if first_image:
                    image_element.screenshot(f"{fig_folder}/{folder_label}_{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.png")
                    first_image = False
            except:
                num_try += 1
                if num_try >= try_max:
                    assert False
                continue
            else:
                break
        
        
        driver.execute_script(f'window.open("{image_src}");')
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        driver.execute_script(f'''
        var a = document.createElement('a');
        a.href = "{image_src}";
        a.download = '';
        document.body.appendChild(a);
        a.click();
        ''')

        time.sleep(4)

        dst_folder = f"{fig_folder}/{folder_label}"
        num_try = 0 
        while True:
            try:
                time.sleep(4)
                img_path = glob.glob("./test/*.png")[0]
                dst_name = f"{start_time.strftime('%Y%m%d')}_{end_time.strftime('%Y%m%d')}.png"
            except:
                num_try += 1
                if num_try >= try_max:
                    assert False
                continue
            else:
                break

        os.makedirs(dst_folder, exist_ok=True)
        os.rename(img_path, f"{dst_folder}/{dst_name}")

        time.sleep(1)
        driver.close()  #링크 이동 후 탭 닫기
        time.sleep(0.25)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(random.randint(2, 10)+30*random.choice([0, 0, 0, 0, 1]))
    
    end_time = start_time - datetime.timedelta(days=1)
    next_end_day = int(end_time.strftime("%d"))


time.sleep(5)
driver.quit()

KeyboardInterrupt: 

In [20]:
driver.quit()